In [1]:
from brainspace.datasets import load_group_fc, load_parcellation, load_conte69
import sys,ossaudiodev
import numpy as np

sys.path.append('/home/romy.beaute/projects/hypnomed/diffusion_embedding/scripts_reordered')
sys.path.append('/mnt/data/romy/hypnomed/git/diffusion_embedding/') 
sys.path.append('/home/romy.beaute/projects/hypnomed/') 
sys.path.append('/mnt/data/romy/hypnomed/git/diffusion_embedding/data/') 
sys.path.append('/home/romy.beaute/projects/hypnomed/diffusion_embedding/data/') 
sys.path.append('/mnt/data/romy/hypnomed/git/diffusion_embedding/vol2surf_derivatives/')

from load_fs import load_fs 
from pipeline import run_perc

f = open('subject_list.txt','r')
var = f.read()
sublist = var.split('\n')[:40]
f.close()
# print(sublist)


In [3]:
# CALCULATE FUNCTIONNAL CONNECTIVITY MATRIX (K) FROM NORMALIZED DATA (loaded from load_fs)


state = 'rs_run-1'
subject=sublist[0]
ses='ses-001'

#Step 1 : load data
K = load_fs(subject, ses, state) #load normalized data
print(K.shape)

(18715, 235)


In [4]:
#Step 2 : Compute correlation coefficient from normalized data K
K[np.isnan(K)] = 0.0

A_mA = K - K.mean(1)[:,None] #Rowwise mean of input arrays & subtract from input arrays themeselves
ssA = (A_mA**2).sum(1)  #Sum of squares across rows
Asq = np.sqrt(np.dot(ssA[:,None],ssA[None]))
Adot = A_mA.dot(A_mA.T)
K = Adot/Asq #finally got K as the correlation coefficient (functional connectivity matrix)

print(K.shape)

(18715, 18715)


## Construction of gradients

In [5]:
from brainspace.gradient import GradientMaps
# help(GradientMaps)

# Ask for 10 gradients (default)
gm = GradientMaps(
    n_components=5,
    approach='dm', #embedding using diffusion maps
    random_state=0)
gm.fit(K) #gradient of connectivity matrix K

/home/romy.beaute/anaconda3/lib/python3.9/site-packages/brainspace/gradient/embedding.py:70: UserWarning: Affinity is not symmetric. Making symmetric.
  warnings.warn('Affinity is not symmetric. Making symmetric.')


GradientMaps(n_components=5, random_state=0)

In [12]:
gm.gradients_.shape 
gm.gradients_[:, 0].shape #1st gradient

(18715, 5)

In [7]:

del A_mA, ssA, Asq, Adot

#Step 3 : Transformation z --> r correlations (scales values b/w [-1;1]
K = run_perc(K, 90) #threshold to keep only top 10% connections

#Step 4 : Compute similarity between all pairs of rows using cosine distance (resulting in positive, symmetrix affinity matrix aff)
norm = (K * K).sum(0, keepdims=True) ** .5 
K = K.T @ K #numerator (dot product)
aff = K / norm / norm.T #similarities (full affinity matrix) shape = (n_samples, n_samples)
# del norm, K

/home/romy.beaute/projects/hypnomed/diffusion_embedding/scripts_reordered/pipeline.py:41: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "run_perc" failed type inference due to: No implementation of function Function(<built-in function setitem>) found for signature:
 
 >>> setitem(array(float32, 2d, C), Tuple(int64, array(bool, 1d, C)), float64)
 
There are 16 candidate implementations:
   - Of which 14 did not match due to:
   Overload of function 'setitem': File: <numerous>: Line N/A.
     With argument(s): '(array(float32, 2d, C), Tuple(int64, array(bool, 1d, C)), float64)':
    No match.
   - Of which 2 did not match due to:
   Overload in function 'SetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 176.
     With argument(s): '(array(float32, 2d, C), Tuple(int64, array(bool, 1d, C)), float64)':
    Rejected as the implementation raised a specific error:
      NumbaNotImplementedError: only one advanced index supp

In [8]:
aff

array([[1.0000007 , 0.6878626 , 0.57642335, ..., 0.03612815, 0.00788714,
        0.        ],
       [0.6878625 , 0.9999991 , 0.8106227 , ..., 0.09799708, 0.03811562,
        0.00349577],
       [0.5764233 , 0.81062263, 1.0000011 , ..., 0.20091775, 0.10557379,
        0.02270358],
       ...,
       [0.03612815, 0.09799708, 0.20091775, ..., 1.0000012 , 0.8942442 ,
        0.8091196 ],
       [0.00788714, 0.03811562, 0.1055738 , ..., 0.89424425, 0.9999999 ,
        0.9115472 ],
       [0.        , 0.00349577, 0.02270358, ..., 0.8091196 , 0.9115471 ,
        1.0000001 ]], dtype=float32)

In [11]:
aff.shape

(18715, 18715)

In [9]:
emb, res = embed.compute_diffusion_map(aff, alpha = 0.5, n_components=5, skip_checks=True, overwrite=True, eigen_solver=eigsh, return_result=True) #Compute the diffusion maps of a symmetric similarity matrix
# del aff

NameError: name 'embed' is not defined

In [ ]:

data = 

FileNotFoundError: No such file or no access: '/mnt/data/romy/hypnomed/git/diffusion_embedding/vol2surf_derivatives/sub-01/ses-001/sub-01_ses-001_task-control.fsa5.lh.mgz'